In [ ]:
!pip install awswrangler

In [ ]:
import pandas as pd 
import awswrangler as wr
from datetime import date

In [ ]:
date = date.today()

In [ ]:
df = wr.athena.read_sql_query("SELECT * FROM sandbox_measurement_analyst.mmix_nvs_xolair_spend", 
                              database="sandbox_measurement_analyst", 
                              workgroup="cmg-oasis-prod-measurement_analyst-wkgrp")

df['npp'] = df['npp'].astype(float).round(0).astype(int)
average_npp = df['npp'].sum() / len(df)

cost_percents = {
    'paid_search_hcp_cost_percent': 0.16,
    'display_hcp_cost_percent': 0.39,
    'paid_social_hcp_cost_percent': 0.22,
    'third_party_email_hcp_cost_percent': 0.06,
    'poc_hcp_cost_percent': 0.08,
    'custom_hcp_cost_percent': 0.08,
    'video_hcp_cost_percent': 0.00,
    'ehr_hcp_cost_percent': 0.01
}
costs = {key: average_npp * percent for key, percent in cost_percents.items()}
cost_df = pd.DataFrame([costs])


wr.s3.to_parquet(
    df=cost_df,
    path=f's3://cmg-oasis-prod-sandbox-measurement-analyst/mmix_nvs_hcp_costs/{date}/',
    dataset=True,
    database="sandbox_measurement_analyst", 
    table="mmix_nvs_hcp_costs",      
    mode="overwrite"
)